In [ ]:
%pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

In [ ]:
import numpy as np
import pandas as pd

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
import os
import cv2
import tensorflow as tf

In [ ]:
with tf.device("/GPU"):
    model = YOLO("yolov8n.yaml")
    results = model.train(data = "../ML_TrafficDetection/data.yaml", epochs = 10)